# LangChain Tutorial: Step-by-Step

Welcome to this step-by-step tutorial on LangChain! We'll go from the basics to more advanced concepts.

## 1. Hello World: Your First LLM Application


In [ ]:
from langchain_openai import ChatOpenAI

# Instantiate the model. We're using chat-gpt-3.5-turbo for this tutorial.
# Make sure your OPENAI_API_KEY is set in your environment variables.
llm = ChatOpenAI(model="gpt-3.5-turbo")

# The invoke method sends a single request to the model.
# The input is a list of messages, in this case, just one from a human.
response = llm.invoke("What is the capital of France?")

print(response.content)


## 2. Prompt Templates: Managing User Input

Hardcoding the prompt like we did above is not very flexible. In a real application, you'll want to take user input and format it into a prompt.

LangChain provides **Prompt Templates** for this. They are recipes for generating prompts and can include instructions, few-shot examples, and user input.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that provides information about countries."),
    ("human", "What is the capital of {country}?"),
])

# We can format the prompt by passing in the variables.
prompt = template.format(country="the United States")

print(prompt)


## 3. Chains: Combining Prompts and Models

So far, we've created a prompt and we have a model. A **Chain** is what lets us combine these two things.

The most common type of chain takes a prompt template, formats it with user input, and then sends the formatted prompt to a model.

LangChain has a special syntax for this called the LangChain Expression Language (LCEL), which uses the pipe `|` operator to chain components together.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that provides information about countries."),
    ("human", "What is the capital of {country}?"),
])

llm = ChatOpenAI(model="gpt-3.5-turbo")

# We create the chain by piping the template to the language model.
chain = template | llm

# We can now invoke the chain with the input variables.
# The chain will first format the prompt and then send it to the LLM.
response = chain.invoke({"country": "Japan"})

print(response.content)


## 4. Agents and Tools: Giving LLMs Access to the World

An **Agent** is a system that uses an LLM as a reasoning engine to decide what actions to take. A **Tool** is a function that the agent can call to get information from the outside world (e.g., a search engine, a calculator, a database).

This is one of the most powerful features of LangChain, as it allows LLMs to interact with external systems and answer questions about data they weren't trained on.

For this example, we'll need to install a new package, `duckduckgo-search`, to use as a tool.


In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai import ChatOpenAI

# 1. Create a tool
search = DuckDuckGoSearchRun()
tools = [search]

# 2. Create an agent
# We're using a prompt from the LangChain Hub. This is a pre-made prompt for a ReAct agent.
prompt = hub.pull("hwchase17/react")
llm = ChatOpenAI(model="gpt-3.5-turbo")
agent = create_react_agent(llm, tools, prompt)

# 3. Create an agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 4. Invoke the agent
response = agent_executor.invoke({
    "input": "Who is the current prime minister of the UK and what is their age?"
})

print(response["output"])


## 5. Memory: Remembering Past Interactions

To build conversational applications like chatbots, your chains and agents need to have **Memory**.

LangChain provides various types of memory. The simplest one is `ConversationBufferMemory`, which just stores the conversation history as a list of messages.

Let's see how to add memory to a chain.


## 6. Output Parsers: Structuring LLM Responses

Often, you want the LLM to return structured data (like JSON, lists, or Python objects) instead of just plain text. **Output Parsers** help you define the structure you want and automatically parse the LLM's output.

Let's see how to use Pydantic-based output parsers to get structured data from an LLM.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# 1. Define the structure you want using Pydantic
class Person(BaseModel):
    name: str = Field(description="The person's name")
    age: int = Field(description="The person's age")
    occupation: str = Field(description="The person's occupation")
    
# 2. Create a model that can output structured data
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Use with_structured_output to make the LLM return structured data
structured_llm = llm.with_structured_output(Person)

# 3. Create a prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at extracting information from text."),
    ("human", "Extract the person's information from the following text: {text}")
])

# 4. Create the chain
chain = prompt | structured_llm

# 5. Invoke with some text
response = chain.invoke({
    "text": "John Smith is a 35-year-old software engineer living in San Francisco."
})

print(f"Name: {response.name}")
print(f"Age: {response.age}")
print(f"Occupation: {response.occupation}")
print(f"Type: {type(response)}")  # This is a Person object!


## 7. RAG (Retrieval Augmented Generation): Teaching LLMs About Your Data

**RAG** is a technique that allows LLMs to answer questions about your own documents or data. It works by:
1. **Splitting** your documents into chunks
2. **Embedding** those chunks (converting them to vectors)
3. **Storing** them in a vector database
4. **Retrieving** relevant chunks when a question is asked
5. **Passing** those chunks to the LLM as context

This is incredibly useful for building chatbots over your documentation, research papers, internal wikis, etc.


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. Create some sample documents
documents = [
    "LangChain is a framework for developing applications powered by language models.",
    "It provides tools for prompt management, chains, agents, and memory.",
    "LangChain was created by Harrison Chase in October 2022.",
    "The framework supports multiple language models including OpenAI, Anthropic, and Hugging Face models.",
    "RAG stands for Retrieval Augmented Generation and is a popular pattern in LangChain.",
]

# 2. Split documents into chunks (in this case, they're already small)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
splits = text_splitter.create_documents(documents)

# 3. Create embeddings and store in a vector database
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embeddings)

# 4. Create a retriever from the vectorstore
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 5. Create a prompt template for RAG
template = """Answer the question based only on the following context:

{context}

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# 6. Create the RAG chain using LCEL
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model="gpt-3.5-turbo")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 7. Ask a question
response = rag_chain.invoke("Who created LangChain?")
print(response)

# Ask another question
response = rag_chain.invoke("What does RAG stand for?")
print(response)


## 8. Streaming: Real-Time Response Generation

For better user experience in chatbots and interactive applications, you often want to **stream** the LLM's response as it's being generated (like ChatGPT does), rather than waiting for the entire response.

LangChain makes streaming easy with the `.stream()` method.


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import sys

# Create a chain
llm = ChatOpenAI(model="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("Tell me a short story about {topic}")
chain = prompt | llm | StrOutputParser()

# Use .stream() instead of .invoke()
print("Streaming response:\n")
for chunk in chain.stream({"topic": "a robot learning to paint"}):
    print(chunk, end="", flush=True)
    
print("\n\nDone!")


In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

# 1. Create a model
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 2. Create a conversation chain
# We set verbose=True to see the prompt being sent to the LLM.
conversation = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    verbose=True
)

# 3. Start the conversation
response = conversation.predict(input="Hi, I'm Bob.")
print(response)

# 4. Continue the conversation
response = conversation.predict(input="What's my name?")
print(response)

# 5. Let's ask another question
response = conversation.predict(input="What's the weather like in London today?")
print(response)
